# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [ ]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np


weather_df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
weather_df = weather_df.sort_values(by=['Date'])
weather_df['Data_Value'] = weather_df['Data_Value']/10
max_df = weather_df.groupby(['Date']).max()
min_df = weather_df.groupby(['Date']).min()

max_temp = max(max_df['Data_Value'])
min_temp = min(min_df['Data_Value'])

max_df = max_df.reset_index()
min_df = min_df.reset_index()
dates = list(map(pd.to_datetime, max_df['Date']))
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for i in range(5,16):
    plt.figure()
    plt.plot(dates, max_df['Data_Value'], '-', c='r')
    plt.plot(dates, min_df['Data_Value'], '-', c='b')
    
    ax = plt.gca()
    
    if i<10:
        ax.axis(['200{}-01-01'.format(i),'200{}-12-31'.format(i), min_temp-1, max_temp+1])
        plt.title('Distribution of highest and lowest temperature in 200{}'.format(i))
    else:
        ax.axis(['20{}-01-01'.format(i),'20{}-12-31'.format(i), min_temp-1, max_temp+1])
        plt.title('Distribution of highest and lowest temperature in 20{}'.format(i))
    
    ax.set_xticklabels(months)
    
    plt.gca().fill_between(dates, 
                           min_df['Data_Value'], max_df['Data_Value'], 
                            facecolor='grey', 
                            alpha=0.25)


    for spine in plt.gca().spines.values():
        spine.set_visible(False)

    x = plt.gca().xaxis

    for item in x.get_ticklabels():
        item.set_rotation(45)
    plt.subplots_adjust(bottom=0.25)
    
    plt.legend(['Max. temp.', 'Min. temp.'], frameon=False, loc=8)
    plt.show()
        


In [ ]:
for i in range(0, len(max_df['Date'])):
    if max_df['Date'][i][5:]=='02-29':
        max_df = max_df.drop(i)
        min_df = min_df.drop(i)

max_df = max_df.reset_index().drop("index", axis=1) 
min_df = min_df.reset_index().drop("index", axis=1) 

max_df

In [ ]:
maxx_df = max_df.drop(["ID", "Element"], axis=1)
minn_df = min_df.drop(["ID", "Element"], axis=1)
max2015_df = max_df.drop(["ID", "Element"], axis=1)
min2015_df = min_df.drop(["ID", "Element"], axis=1)


for i in range(0, len(max_df['Date'])):
    if max_df['Date'][i][:4]=='2015':
        maxx_df = maxx_df.drop(i)
        
        minn_df = minn_df.drop(i)
        
    else:
        max2015_df = max2015_df.drop(i)
        
        min2015_df = min2015_df.drop(i)
        

max2015_df['Date'] = max2015_df['Date'].str.slice(5)
min2015_df['Date'] = min2015_df['Date'].str.slice(5)
maxx_df['Date'] = maxx_df['Date'].str.slice(5)
minn_df['Date'] = minn_df['Date'].str.slice(5)
        
maxx_df = maxx_df.groupby(['Date']).max().reset_index()
minn_df = minn_df.groupby(['Date']).min().reset_index()
max2015_df = max2015_df.reset_index().drop("index", axis=1)
min2015_df = min2015_df.reset_index().drop("index", axis=1) 

maxx_df['Date'] = range(1,366)
minn_df['Date'] = range(1,366)
max2015_df['Date'] = range(1,366)
min2015_df['Date'] = range(1,366)

In [ ]:
x1 = []
y1 = []

x2 = []
y2 = []

for i in range(0, len(max2015_df['Date'])):
    if max2015_df['Data_Value'][i] > maxx_df['Data_Value'][i]:
        y1.append(max2015_df['Data_Value'][i])
        x1.append(max2015_df['Date'][i])
    if min2015_df['Data_Value'][i] < minn_df['Data_Value'][i]:
        y2.append(max2015_df['Data_Value'][i])
        x2.append(max2015_df['Date'][i])
            

            
plt.figure()
plt.scatter(x1, y1, c='r')
plt.scatter(x2, y2, c='b')

plt.title('Days for which the ten year record (2005-2014) record high or record low temperature was broken in 2015')
plt.legend(['High temp.', 'Low temp.'], frameon=False, loc=4)